In [60]:
def associer_cluster_classes(y_pred, y_test):
  """
  En entrée : la liste des prédictions (y_pred) et la liste des étiquettes attendues (y_test)
  En sortie : une dictionnaire qui associe chaque cluster la classe qui amximise les vrais positifs
  """
  import numpy
  from sklearn.metrics.cluster import contingency_matrix
  matrice1 = contingency_matrix(y_pred, y_test)
  matrice =  [numpy.ndarray.tolist(l) for l in matrice1]# pour manipuler la matrice comme une liste de liste
  dic_clusters = {}
  while len(dic_clusters)<len(matrice):#tant qu'on n'a pas associé tous les clusters à une classe (y_test)
    for cpt, ligne in enumerate(matrice):
      m = max(ligne)# on cherche la plus grande valeur pour chaque cluster
      if m == numpy.amax(matrice):#si cette valeur correspond à la plus grande valeur de la matrice
        if m==0:#la matrice est vide, on associe les clusters restants aux classes restantes
          clusters_left = [x for x in range(len(matrice)) if x not in dic_clusters.keys()]
          classes_left = [x for x in range(len(matrice)) if x not in dic_clusters.values()]
          for cpt, x in enumerate(clusters_left):
            dic_clusters[x] = classes_left[cpt]
        pos = ligne.index(m)
        if cpt not in dic_clusters:#et si on a pas encore associé ce cluster à une classe
          dic_clusters[cpt] = pos
          matrice[cpt] = [0 for x in matrice[cpt]]#on "efface" la ligne, le cluster est désormais associé à une classe
          for i in range(len(matrice)):
            matrice[i][pos]=0 #on efface aussi la colonne, la classe est désormais associée à un cluster
  return dic_clusters, matrice1

In [63]:
def evaluer_clustering(nom_classes, y_pred, y_test):
  """
  On va évaluer le clustering pour chaque classe
  En entrée : le nom des classes, la prédiction et la vérité de terrain
  En sortie : pour chaque classe, Rappel Précision et F-mesure + association clusters et classes
  """
  # Pour chaque nom de classe on donne un identifiant numérique
  dic_classes = {classe:cpt for cpt, classe in enumerate(nom_classes)}
  # Avec cet identifiant, on "traduit" le nom de la classe
  y_test = [dic_classes[x] for x in y_test]
  #Come cela on peut comparer la prédiction et la vérité (toutes deux sont sous forme de chiffres)
  dic_clusters, matrice= associer_cluster_classes(y_pred, y_test)
  dic= {}# Pour passer à l'évaluation proprement dite :
  for num_cluster, ligne in enumerate(matrice):
    num_classe = dic_clusters[num_cluster]#la classe associée au cluster
    VP = ligne[num_classe]#le nombre d'élemenst en commun entre la classe et le cluster
    FP = sum(ligne)-VP #le reste de la ligne ce sont des faux positifs -> ne devraient pas être dans ce cluster
    L =[ x[num_classe] for x in matrice]
    FN = sum(L)-VP#le reste de la colone ce sont des faux négatifs -> devraient être dans ce cluster
    if VP!=0:
      R = VP/(VP+FN)
      P = VP/(VP+FP)
      beta = 1
      F = (1+beta*beta)*(P*R)/((beta*beta*P)+R)
    else:
      R, P, F =0, 0, 0
    resultats = {"Rappel":round(R, 4), "Précision":round(P, 4), "F-mesure":round(F,4)}
    dic[nom_classes[num_classe]] = resultats 
  # On transforme dic_clusters pour donner les vrais noms des classes en sortie
  dic_clusters= {num_cluster : nom_classes[num_classe] for num_cluster, num_classe in dic_clusters.items()}
  return {"Résultats":dic, "dic_clusters":dic_clusters}


In [64]:
## la liste des noms de classes:
classes = ["A", "B", "C", "D"]

##y_test la liste des étiquettes de chaque document
y_test = ['A', 'D', 'B', 'B', 'D', 'D', 'B', 'B', 'B', 'B', 'A', "C", "C"]

##y_pred : la liste des clusters attrivués à chaque document
y_pred = [  1,   2,   3,   3,   1,   1,   2,   2,   3,   2,   0,   1,   1]

## dico donne pour chaque classe l'évaluation ("Résultats") et associe clusters et classes
dico = evaluer_clustering(classes, y_pred, y_test)
print(dico["dic_clusters"])
for nom_classe, res in dico["Résultats"].items():
    print(nom_classe, res)

{2: 'B', 1: 'C', 0: 'A', 3: 'D'}
A {'Rappel': 0.5, 'Précision': 1.0, 'F-mesure': 0.6667}
C {'Rappel': 1.0, 'Précision': 0.4, 'F-mesure': 0.5714}
B {'Rappel': 0.5, 'Précision': 0.75, 'F-mesure': 0.6}
D {'Rappel': 0, 'Précision': 0, 'F-mesure': 0}
